<a href="https://colab.research.google.com/github/SwaksharDeb/Activity-recognition/blob/master/HAR_paper1_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from math import sqrt
import math

In [ ]:
"""Import Datasets"""
dataset_user04 = pd.read_csv("final_field_user04_dataset.csv")
dataset_user07 = pd.read_csv("final_field_user07_dataset.csv")
dataset_user08 = pd.read_csv("final_field_user08_dataset.csv")
dataset_user18 = pd.read_csv("final_field_user18_dataset.csv")
dataset_user38 = pd.read_csv("final_field_user38_dataset.csv")
dataset_user51 = pd.read_csv("final_field_user51_dataset.csv")
dataset_lab = pd.read_csv("lab data localized.csv")

dataset = pd.concat([dataset_lab, dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user38, dataset_user51], ignore_index = True)


In [ ]:
""" Data processing"""
X = dataset.iloc[:, [0, 1, 2]].values
Y = dataset.iloc[:, 3].values
Y = Y - 1

In [ ]:
low_pass_filter_alpha8 = np.zeros((X.shape[0], X.shape[1]))
high_pass_filter_alpha8 = np.zeros((X.shape[0], X.shape[1]))
low_pass_filter_alpha83 = np.zeros((X.shape[0], X.shape[1]))
low_pass_filter_alpha86 = np.zeros((X.shape[0], X.shape[1]))

low_pass_filter_alpha6 = np.zeros((X.shape[0], X.shape[1]))
high_pass_filter_alpha6 = np.zeros((X.shape[0], X.shape[1]))
low_pass_filter_alpha63 = np.zeros((X.shape[0], X.shape[1]))
low_pass_filter_alpha66 = np.zeros((X.shape[0], X.shape[1]))

low_pass_filter_alpha7 = np.zeros((X.shape[0], X.shape[1]))
high_pass_filter_alpha7 = np.zeros((X.shape[0], X.shape[1]))
low_pass_filter_alpha73 = np.zeros((X.shape[0], X.shape[1]))
low_pass_filter_alpha76 = np.zeros((X.shape[0], X.shape[1]))

low_pass_filter_alpha5 = np.zeros((X.shape[0], X.shape[1]))
low_pass_filter_alpha53 = np.zeros((X.shape[0], X.shape[1]))
low_pass_filter_alpha56 = np.zeros((X.shape[0], X.shape[1]))

low_pass_filter_alpha9 = np.zeros((X.shape[0], X.shape[1]))
low_pass_filter_alpha93 = np.zeros((X.shape[0], X.shape[1]))
low_pass_filter_alpha96 = np.zeros((X.shape[0], X.shape[1]))

alpha = [0.5,0.53,0.56,0.6,0.63,0.66,0.7,0.73,0.76,0.8,0.83,0.86,0.9,0.93,0.96]

for x in alpha:
    for i in range(len(X)):
        for j in range(X.shape[1]):
            if i == 0:
                low_pass_filter_alpha8[i][j] = X[i][j]
                high_pass_filter_alpha8[i][j] = X[i][j]
            elif x==0.5 and i!=j:
                low_pass_filter_alpha5[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
                #high_pass_filter_alpha5[i][j] = X[i][j] - low_pass_filter_alpha8[i][j]
            elif x==0.53 and i!=j:
                low_pass_filter_alpha53[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.56 and i!=j:
                low_pass_filter_alpha56[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.6 and i!=j:
                low_pass_filter_alpha6[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.63 and i!=j:
                low_pass_filter_alpha63[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.66 and i!=j:
                low_pass_filter_alpha66[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.7 and i!=j:
                low_pass_filter_alpha7[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.73 and i!=j:
                low_pass_filter_alpha73[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.76 and i!=j:
                low_pass_filter_alpha76[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.8 and i!=j:
                low_pass_filter_alpha8[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.83 and i!=j:
                low_pass_filter_alpha83[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.86 and i!=j:
                low_pass_filter_alpha86[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.9 and i!=j:
                low_pass_filter_alpha9[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            elif x==0.93 and i!=j:
                low_pass_filter_alpha93[i][j] = x*X[i-1][j] + (1-x)*X[i][j]
            else:
                low_pass_filter_alpha96[i][j] = x*X[i-1][j] + (1-x)*X[i][j]

In [ ]:
# frame preparation
Fs = 4
frame_size = Fs*20 # 80 # 20 seconds of data
hop_size =  Fs*10

def get_frames(df, frame_size, hop_size,label_data):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        value = df[i: i + frame_size, :]
        #y = df[i: i + frame_size, 1]
        #z = df[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(label_data[i: i + frame_size])[0][0]
        frames.append([value])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

row_frame, row_label = get_frames(X, frame_size, hop_size, Y)
low_pass_frame_alpha8, low_pass_label_alpha8 = get_frames(low_pass_filter_alpha8, frame_size, hop_size, Y)
high_pass_frame_alpha8, high_pass_label_alpha8 = get_frames(high_pass_filter_alpha8, frame_size, hop_size, Y)
low_pass_frame_alpha7, low_pass_label_alpha7 = get_frames(low_pass_filter_alpha7, frame_size, hop_size, Y)
high_pass_frame_alpha7, high_pass_label_alpha7 = get_frames(high_pass_filter_alpha7, frame_size, hop_size, Y)
low_pass_frame_alpha6, low_pass_label_alpha6 = get_frames(low_pass_filter_alpha6, frame_size, hop_size, Y)
high_pass_frame_alpha6, high_pass_label_alpha6 = get_frames(high_pass_filter_alpha6, frame_size, hop_size, Y)
low_pass_frame_alpha63, low_pass_label_alpha63 = get_frames(low_pass_filter_alpha63, frame_size, hop_size, Y)
low_pass_frame_alpha66, low_pass_label_alpha66 = get_frames(low_pass_filter_alpha66, frame_size, hop_size, Y)
low_pass_frame_alpha5, low_pass_label_alpha5 = get_frames(low_pass_filter_alpha5, frame_size, hop_size, Y)
low_pass_frame_alpha53, low_pass_label_alpha53 = get_frames(low_pass_filter_alpha53, frame_size, hop_size, Y)
low_pass_frame_alpha56, low_pass_label_alpha56 = get_frames(low_pass_filter_alpha56, frame_size, hop_size, Y)
low_pass_frame_alpha73, low_pass_label_alpha73 = get_frames(low_pass_filter_alpha73, frame_size, hop_size, Y)
low_pass_frame_alpha76, low_pass_label_alpha76 = get_frames(low_pass_filter_alpha76, frame_size, hop_size, Y)
low_pass_frame_alpha83, low_pass_label_alpha83 = get_frames(low_pass_filter_alpha83, frame_size, hop_size, Y)
low_pass_frame_alpha86, low_pass_label_alpha86 = get_frames(low_pass_filter_alpha86, frame_size, hop_size, Y)
low_pass_frame_alpha9, low_pass_label_alpha9 = get_frames(low_pass_filter_alpha9, frame_size, hop_size, Y)
low_pass_frame_alpha93, low_pass_label_alpha93 = get_frames(low_pass_filter_alpha93, frame_size, hop_size, Y)
low_pass_frame_alpha96, low_pass_label_alpha96 = get_frames(low_pass_filter_alpha96, frame_size, hop_size, Y)

In [ ]:
low_pass_frame = np.concatenate((low_pass_frame_alpha5,low_pass_frame_alpha53,low_pass_frame_alpha56,low_pass_frame_alpha6,low_pass_frame_alpha63,low_pass_frame_alpha66,low_pass_frame_alpha7,low_pass_frame_alpha73,low_pass_frame_alpha76,low_pass_frame_alpha8,low_pass_frame_alpha83,low_pass_frame_alpha86,low_pass_frame_alpha9,low_pass_frame_alpha93,low_pass_frame_alpha96))

high_pass_frame = np.concatenate((high_pass_frame_alpha8,high_pass_frame_alpha7,high_pass_frame_alpha6))

low_pass_label = np.concatenate((low_pass_label_alpha5,low_pass_label_alpha53,low_pass_label_alpha56,low_pass_label_alpha6,low_pass_label_alpha63,low_pass_label_alpha66,low_pass_label_alpha7,low_pass_label_alpha73,low_pass_label_alpha76,low_pass_label_alpha8,low_pass_label_alpha83,low_pass_label_alpha86,low_pass_label_alpha9,low_pass_label_alpha93,low_pass_label_alpha96))

high_pass_label = np.concatenate((high_pass_label_alpha8,high_pass_label_alpha7,high_pass_label_alpha6))


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(row_frame, row_label, test_size = 0.2, random_state = 0, stratify = row_label)

In [ ]:
# Data Augmentation
scalling_factor = [0.3,0.5,0.7,0.8,1.2,1.5,2]

for scal in scalling_factor:
    X_train_1_scal = X_train[np.where(y_train==1)]*scal
    label_1 = y_train[np.where(y_train==1)]
    X_train_5_scal = X_train[np.where(y_train==5)]*scal
    label_5 = y_train[np.where(y_train==5)]
    X_train_7_scal = X_train[np.where(y_train==7)]*scal
    label_7 = y_train[np.where(y_train==7)]
    X_train_8_scal = X_train[np.where(y_train==8)]*scal
    label_8 = y_train[np.where(y_train==8)]
    X_train_9_scal = X_train[np.where(y_train==9)]*scal
    label_9 = y_train[np.where(y_train==9)]
    X_train_10_scal = X_train[np.where(y_train==10)]*scal
    label_10 = y_train[np.where(y_train==10)]
    X_train = np.concatenate((X_train, X_train_1_scal,X_train_5_scal,X_train_7_scal,X_train_8_scal,X_train_9_scal,X_train_10_scal))
    y_train = np.concatenate((y_train, label_1, label_5, label_7, label_8, label_9, label_10))

scalling_factor = [0.5,2]

for scal in scalling_factor:
    X_train_0_scal = X_train[np.where(y_train==0)]*scal
    label_0 = y_train[np.where(y_train==0)]
    X_train_3_scal = X_train[np.where(y_train==3)]*scal
    label_3 = y_train[np.where(y_train==3)]
    X_train = np.concatenate((X_train, X_train_0_scal, X_train_3_scal))
    y_train = np.concatenate((y_train, label_0, label_3))

In [ ]:
low_pass_frame_6 = low_pass_frame[np.where(low_pass_label==6)]
X_train = np.concatenate((X_train,low_pass_frame_6))
low_pass_frame_0 = low_pass_frame[np.where(low_pass_label==0)]
X_train = np.concatenate((X_train,low_pass_frame_0))
low_pass_frame_1 = low_pass_frame[np.where(low_pass_label==1)]
X_train = np.concatenate((X_train,low_pass_frame_1))
low_pass_frame_2 = low_pass_frame[np.where(low_pass_label==2)]
X_train = np.concatenate((X_train,low_pass_frame_2))
low_pass_frame_7 = low_pass_frame[np.where(low_pass_label==7)]
X_train = np.concatenate((X_train,low_pass_frame_7))
low_pass_frame_8 = low_pass_frame[np.where(low_pass_label==8)]
X_train = np.concatenate((X_train,low_pass_frame_8))
low_pass_frame_5 = low_pass_frame[np.where(low_pass_label==5)]
X_train = np.concatenate((X_train,low_pass_frame_5))
low_pass_frame_9 = low_pass_frame[np.where(low_pass_label==9)]
X_train = np.concatenate((X_train,low_pass_frame_9))
low_pass_frame_10 = low_pass_frame[np.where(low_pass_label==10)]
X_train = np.concatenate((X_train,low_pass_frame_10))
low_pass_frame_11 = low_pass_frame[np.where(low_pass_label==11)]
X_train = np.concatenate((X_train,low_pass_frame_11))


In [ ]:
low_pass_label_6 = low_pass_label[np.where(low_pass_label==6)]
y_train = np.concatenate((y_train,low_pass_label_6))
low_pass_label_0 = low_pass_label[np.where(low_pass_label==0)]
y_train = np.concatenate((y_train,low_pass_label_0))
low_pass_label_1 = low_pass_label[np.where(low_pass_label==1)]
y_train = np.concatenate((y_train,low_pass_label_1))
low_pass_label_2 = low_pass_label[np.where(low_pass_label==2)]
y_train = np.concatenate((y_train,low_pass_label_2))
low_pass_label_7 = low_pass_label[np.where(low_pass_label==7)]
y_train = np.concatenate((y_train,low_pass_label_7))
low_pass_label_8 = low_pass_label[np.where(low_pass_label==8)]
y_train = np.concatenate((y_train,low_pass_label_8))
low_pass_label_5 = low_pass_label[np.where(low_pass_label==5)]
y_train = np.concatenate((y_train,low_pass_label_5))
low_pass_label_9 = low_pass_label[np.where(low_pass_label==9)]
y_train = np.concatenate((y_train,low_pass_label_9))
low_pass_label_10 = low_pass_label[np.where(low_pass_label==10)]
y_train = np.concatenate((y_train,low_pass_label_10))
low_pass_label_11 = low_pass_label[np.where(low_pass_label==11)]
y_train = np.concatenate((y_train,low_pass_label_11))

In [ ]:
np.where(np.all(X_train==0, axis=2))

(array([270368, 271064, 271760, 272456, 273152, 273848, 274544, 275240,
        275936, 277328, 278024, 278720, 279416, 280112]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
X_train.shape, y_train.shape

((364838, 80, 3), (364838,))

In [ ]:
y_train = np.delete(y_train, np.where(~X_train.any(axis=2))[0], axis=0)
X_train = np.delete(X_train, np.where(~X_train.any(axis=2))[0], axis=0)

In [ ]:
X_train.shape, y_train.shape

((87561105,), (364823,))

In [ ]:
Counter(y_train)

Counter({0.0: 12668,
         1.0: 29937,
         2.0: 41617,
         3.0: 33432,
         4.0: 21640,
         5.0: 65157,
         6.0: 45773,
         7.0: 18197,
         8.0: 26859,
         9.0: 24955,
         10.0: 23036,
         11.0: 21567})

In [ ]:
np.where(np.all(X_train==0, axis=2))

(array([], dtype=int64), array([], dtype=int64))

In [ ]:
#feature calculation
mean = np.mean(X_train, axis = 1)  # mean along (x,y,z) axis
std = np.std(X_train, axis = 1)  # standard deviation along (x,y,z) axis

rms_x = []  # root mean square along x axis
value = 0
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        value = value + (X_train[i][j][0] **2)
        value = value / (X_train.shape[1])
    value = sqrt(value)
    rms_x.append(value)
    value = 0

rms_y = []  # root mean square along y axis
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        value = value + (X_train[i][j][1] **2)
        value = value / (X_train.shape[1])
    value = sqrt(value)
    rms_y.append(value)
    value = 0

rms_z = []  # root mean square z axis
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        value = value + (X_train[i][j][2] **2)
        value = value / (X_train.shape[1])
    value = sqrt(value)
    rms_z.append(value)
    value = 0

lengths = []  # length of the accelerometer vector
length = 0
nx, ny, nz = X_train.shape[0], X_train.shape[1], X_train.shape[2]
acc_vector = np.reshape(X_train,(nx, ny*nz))

for i in range(acc_vector.shape[0]):
    count = 0
    for j in range(acc_vector.shape[1]):
        length += acc_vector[i][j]**2
        count += 1

        if count == 3:
            length = sqrt(length)
            lengths.append(length)
            length = 0
            count = 0

avc = []  # avc feature

polling_rate = 4 #Hz

for i in range(0, len(lengths), frame_size):
    value = 0
    iterable = lengths[i:i+frame_size]
    for j in range(len(iterable) - 1, 1, -1):
        value += abs(iterable[j] - iterable[j - 1])

    value = value / (frame_size / polling_rate)
    avc.append(value)

max_min = []

for i in range(0, len(lengths), frame_size):
    iterable = lengths[i:i+frame_size]

    max_min.append(max(iterable) - min(iterable))

angle = np.zeros((X_train.shape[0],X_train.shape[1],X_train.shape[2]))  # inclenation angle
angles = []
length = 0
for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        length = X_train[i,j,0]**2 + X_train[i,j,1]**2 + X_train[i,j,2]**2
        length = sqrt(length)
        if length == 0:
            print("length is zero in index row= ",i,"column= ",j)
        else:
            angle_x = math.acos(X_train[i,j,0]/length)
            angle_y = math.acos(X_train[i,j,1]/length)
            angle_z = math.acos(X_train[i,j,2]/length)
            angle[i][j][0] = angle_x
            angle[i][j][1] = angle_y
            angle[i][j][2] = angle_z
nx, ny, nz = angle.shape
angle = np.reshape(angle,(nx,ny*nz))

input_vector = np.zeros((X_train.shape[0], angle.shape[1] + 11))  # input vector for the algorithm
for i in range(input_vector.shape[0]):
    input_vector[i][0] = mean[i][0]
    input_vector[i][1] = mean[i][1]
    input_vector[i][2] = mean[i][2]
    input_vector[i][3] = std[i][0]
    input_vector[i][4] = std[i][1]
    input_vector[i][5] = std[i][2]
    input_vector[i][6] = rms_x[i]
    input_vector[i][7] = rms_y[i]
    input_vector[i][8] = rms_z[i]
    input_vector[i][9] = max_min[i]
    input_vector[i][10] = avc[i]

for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        input_vector[i][j+11] = angle[i][j]

In [ ]:
# for test set
mean = np.mean(X_val, axis = 1)  # mean along (x,y,z) axis
std = np.std(X_val, axis = 1)  # standard deviation along (x,y,z) axis

rms_x = []  # root mean square along x axis
value = 0
for i in range(X_val.shape[0]):
    for j in range(X_val.shape[1]):
        value = value + (X_val[i][j][0] **2)
        value = value / (X_val.shape[1])
    value = sqrt(value)
    rms_x.append(value)
    value = 0

rms_y = []  # root mean square along y axis
for i in range(X_val.shape[0]):
    for j in range(X_val.shape[1]):
        value = value + (X_val[i][j][1] **2)
        value = value / (X_val.shape[1])
    value = sqrt(value)
    rms_y.append(value)
    value = 0

rms_z = []  # root mean square z axis
for i in range(X_val.shape[0]):
    for j in range(X_val.shape[1]):
        value = value + (X_val[i][j][2] **2)
        value = value / (X_val.shape[1])
    value = sqrt(value)
    rms_z.append(value)
    value = 0

lengths = []  # length of the accelerometer vector
length = 0
nx, ny, nz = X_val.shape[0], X_val.shape[1], X_val.shape[2]
acc_vector = np.reshape(X_val,(nx, ny*nz))

for i in range(acc_vector.shape[0]):
    count = 0
    for j in range(acc_vector.shape[1]):
        length += acc_vector[i][j]**2
        count += 1

        if count == 3:
            length = sqrt(length)
            lengths.append(length)
            length = 0
            count = 0

avc = []  # avc feature

polling_rate = 4 #Hz

for i in range(0, len(lengths), frame_size):
    value = 0
    iterable = lengths[i:i+frame_size]
    for j in range(len(iterable) - 1, 1, -1):
        value += abs(iterable[j] - iterable[j - 1])

    value = value / (frame_size / polling_rate)
    avc.append(value)

max_min = []

for i in range(0, len(lengths), frame_size):
    iterable = lengths[i:i+frame_size]

    max_min.append(max(iterable) - min(iterable))

angle = np.zeros((X_val.shape[0],X_val.shape[1],X_val.shape[2]))  # inclenation angle
angles = []
length = 0
for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        length = X_val[i,j,0]**2 + X_val[i,j,1]**2 + X_val[i,j,2]**2
        length = sqrt(length)
        if length == 0:
            print("length is zero in index row= ",i,"column= ",j)
        else:
            angle_x = math.acos(X_val[i,j,0]/length)
            angle_y = math.acos(X_val[i,j,1]/length)
            angle_z = math.acos(X_val[i,j,2]/length)
            angle[i][j][0] = angle_x
            angle[i][j][1] = angle_y
            angle[i][j][2] = angle_z
nx, ny, nz = angle.shape
angle = np.reshape(angle,(nx,ny*nz))

X_val = np.zeros((X_val.shape[0], angle.shape[1] + 11))  # input vector for the algorithm
for i in range(X_val.shape[0]):
    X_val[i][0] = mean[i][0]
    X_val[i][1] = mean[i][1]
    X_val[i][2] = mean[i][2]
    X_val[i][3] = std[i][0]
    X_val[i][4] = std[i][1]
    X_val[i][5] = std[i][2]
    X_val[i][6] = rms_x[i]
    X_val[i][7] = rms_y[i]
    X_val[i][8] = rms_z[i]
    X_val[i][9] = max_min[i]
    X_val[i][10] = avc[i]

for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        X_val[i][j+11] = angle[i][j]


In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((348174, 80, 3), (364838,), (9339, 251), (9339,))

In [ ]:
sc = StandardScaler()
input_vector = sc.fit_transform(input_vector)
X_val = sc.transform(X_val)

In [ ]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1], )))
#model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(units=324, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=12, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 50, validation_data= (X_val, y_val), verbose=1)

In [ ]:
y_pred = model.predict(X_val)

In [ ]:
y_pred_ = np.zeros((y_pred.shape[0]))
for i in range(y_pred.shape[0]):
    y_pred_[i] = np.argmax(y_pred[i,:])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred_, y_val)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred)

In [ ]:
per_class_accuracy = []
for i in range(cm.shape[0]):
    accuracy = cm[i,i]/sum(cm[i,:])
    per_class_accuracy.append(accuracy)